This is the notebook with the code and procedure when you want to use the trained model on new data

In [1]:
!pip install rdkit-pypi
!pip install deepchem
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install dgllife
!pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 KB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 24.1 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
DEPRECATION: The HTML index page being used (https://data.dgl.ai/wheels/cu118/repo.html) is not a proper HTML 5 document. This is in violation of PEP 503 which requires these pages to be well-formed HTML 5 documents. Please reach out to the owners of this index page, and ask them 

In [2]:
import deepchem as dc
from rdkit import Chem
import networkx as nx
import numpy as np
import pandas as pd
import random

# Set the seed
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
data_path = 'data/data_from_42_filtered.csv' #'path_to/data/your_data'
dataset = pd.read_csv(data_path,encoding='unicode_escape')
dataset

,Unnamed: 0,Name,Smile,avg.exp.value(cal/cc)1/2
0,0,"1,3- butadien",C=CC=CÃÂ ÃÂ,7.10
1,1,"1,3-BUTANEDIOL",CC(CCO)O,14.14
2,3,1-Chlorobutane,ÃÂ CCCCClÃÂ ÃÂ,8.44
3,4,1-Pentanol,CCCCCOÃÂ ÃÂ,10.60
4,6,2-Ethyl-1-hexanol,CCCCC(CC)COÃÂ ÃÂ,9.85
...,...,...,...,...
57,59,Succinic anhydride,C1CC(=O)OC1=OÃÂ,15.40
58,60,Tetrahydrofuran,C1CCOC1ÃÂ ÃÂ,9.10
59,61,Tetrahydronaphthalene,C1CCC2=CC=CC=C2C1ÃÂ ÃÂ,9.60
60,62,Toluene,CC1=CC=CC=C1ÃÂ ÃÂ,8.94


In [4]:
smiles=dataset['Smile'].to_list()
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
out = featurizer.featurize(smiles)

In [5]:
x = dc.data.NumpyDataset(out)

In [ ]:
from deepchem.models import AttentiveFPModel,GATModel
# Unfortumatelly you can't load the models if you do not give it a state dictionary.
#These are the ones from our paper but if you wan't to use your own just modify the code below

params={'dropout': 0.43624502513391994, 'learning_rate': 0.0003944038911898053, 'num_layers': 2, 'graph_feat_size': 300, 'num_timesteps': 8}

D_model = AttentiveFPModel(n_tasks=1,**params,model_dir="/trained_models/gnn/GAT_D_final_model") 
D_model.restore()

params = {'dropout': 0.4236010960038035, 'learning_rate': 0.000987802242200037, 'graph_attention_layers': [64, 64, 64]}
P_model = GATModel(n_tasks=1,**params,model_dir="/trained_models/gnn/GAT_D_final_model") 
P_model.restore()

params={'dropout': 0.21798085672442016, 'learning_rate': 0.000647424245954023, 'num_layers': 2, 'graph_feat_size': 300, 'num_timesteps': 8}
H_model = AttentiveFPModel(n_tasks=1,**params,model_dir="/trained_models/gnn/GAT_D_final_model") 
H_model.restore()

In [ ]:
y_D = D_model.predict(x)
y_H = H_model.predict(x)
y_P = P_model.predict(x)

dataset['D predicted'] = y_D
dataset['P predicted'] = y_P
dataset['H predicted'] = y_H

dataset['PRED SOLUBILITY^2'] = y_D**2 + y_H**2 + y_P**2
dataset['PRED SOLUBILITY'] = np.sqrt(dataset['PRED SOLUBILITY^2'])
#for ref 42 data:
#df['PREDICTED SOL (cal/cc)1/2'] = df['PREDICTED SOL']/2.0455
dataset

,Name,Smile,total,H,P,D,D predicted,P predicted,H predicted,PRED SOLUBILITY^2,PRED SOLUBILITY
0,PROPANE,CCC,13.10,0.00,0.00,13.100000,13.560548,0.417874,-0.032759,184.064148,13.567024
1,BUTANE,CCCC,14.10,0.00,0.00,14.100000,13.533257,0.316725,-0.036226,183.250687,13.537012
2,PENTANE,CCCCC,14.40,0.00,0.00,14.400000,14.141557,0.334942,-0.042892,200.097656,14.145588
3,HEXANE,CCCCCC,14.90,0.00,0.00,14.900000,14.877765,0.025709,-0.054575,221.351517,14.877887
4,HEPTANE,CCCCCCC,15.20,0.00,0.00,15.200000,15.150198,0.031562,-0.073685,229.534927,15.150410
5,octane,CCCCCCCC,15.40,0.00,0.00,15.400000,15.285212,0.029540,-0.080729,233.645081,15.285453
6,NONANE,CCCCCCCCC,15.60,0.00,0.00,15.600000,15.356138,0.043562,-0.075281,235.818527,15.356384
7,DECANE,CCCCCCCCCC,15.70,0.00,0.00,15.700000,15.442492,0.036789,-0.064526,238.476105,15.442672
8,Undecane,CCCCCCCCCCC,15.80,0.00,0.00,15.800000,15.580613,0.042071,-0.068286,242.761948,15.580820
9,DODECANE,CCCCCCCCCCCC,15.90,0.00,0.00,15.900000,15.742816,0.048727,-0.086228,247.846069,15.743128


In [13]:
#Save the prediction data
dataset.to_csv('results/gnn/data_from_6_filtered_predictions.csv')#'path_to_save_folder/GNNpredictions.csv')